In [1]:
import os
import torch
import numpy as np
import time
import random
from sklearn.model_selection import KFold
import argparse
import timeit
from dataset import prepare_data
import psutil
import torch
from models.model_rbgm import GNN_1
from memory_capacity_utils import gen_lag_data, compute_memory_capacity_vectorized, get_mem_cap_from_model

running on GPU


In [2]:
if torch.cuda.is_available():
    device = torch.device('cuda:0')
    print('running on GPU')
else:
    device = torch.device("cpu")
    print('running on CPU')

running on GPU


# Load the dataset

In [23]:
def get_args():
    parser = argparse.ArgumentParser(description='Args for graph predition')
    parser.add_argument('-num_folds', type=int, default=5, help='cv number')
    parser.add_argument('--num_timepoints', type=int, default=3,
                        help='Number of timepoints')
    parser.add_argument('-num_epochs', type=int, default=20, help='number of epochs')
    parser.add_argument('--lr', type=float, default=0.001, help="Learning rate")
    parser.add_argument('--memcap_coef', type=float, default=0.0005, help="Memory Capacity Loss Coefficient")
    parser.add_argument('-max_lag', type=int, default=35, help='Lag tao for memory capacity signals')
    parser.add_argument('-save_path',type=str,default = '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/',help='Path to the saved results')
    args, _ = parser.parse_known_args()
    return args

In [24]:
args = get_args()

In [25]:
dataset = np.load("datasets/multivariate_simulation_data_2.npy")
dataset = torch.from_numpy(dataset).squeeze()
dataset = dataset.type(torch.FloatTensor)


# Set up the network architecture and train-validate

**Train-validate-functions**

In [18]:
def create_directory_if_not_exists(directory):
    """
    Checks if a specified directory exists, and creates it if it doesn't.

    Args:
    - directory (str): Path of the directory to check and potentially create.
    """
    if not os.path.exists(directory):
        os.makedirs(directory)
        print(f"Directory '{directory}' was created.")
    else:
        print(f"Directory '{directory}' already exists.")

# Create the results folders
create_directory_if_not_exists(args.save_path+'real_and_predicted_graphs')
create_directory_if_not_exists(args.save_path+'train_losses/mae_losses')
create_directory_if_not_exists(args.save_path+'train_losses/reservoir_losses')
create_directory_if_not_exists(args.save_path+'train_losses/bio_losses')
create_directory_if_not_exists(args.save_path+'train_losses/tp_losses')
create_directory_if_not_exists(args.save_path+'train_losses/total_losses')
create_directory_if_not_exists(args.save_path+'test_mae_losses')
create_directory_if_not_exists(args.save_path+'test_tp_losses')
create_directory_if_not_exists(args.save_path+'test_memcap_losses')
create_directory_if_not_exists(args.save_path+'test_predicted')
create_directory_if_not_exists(args.save_path+'test_original')
create_directory_if_not_exists(args.save_path+'trained_models')

Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/real_and_predicted_graphs' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/train_losses/mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/train_losses/reservoir_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/train_losses/bio_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/train_losses/tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/train_losses/total_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/test_mae_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/test_tp_losses' already exists.
Directory '/vol/bitbucket/sx420/4D-FedGNN-Plus/results/reservoir_rbgm/test_memcap_losses' already exists.
Directory '/vo

In [19]:
manual_seed = 777
np.random.seed(manual_seed)
random.seed(manual_seed)
torch.manual_seed(manual_seed)

# Reservoir signals
X_train_res_np, y_train_res_np = gen_lag_data(1000, 41, args.max_lag)
X_test_res_np, y_test_res_np = gen_lag_data(500, 42, args.max_lag)
X_train_res = torch.from_numpy(X_train_res_np).unsqueeze(1).to(device, dtype=torch.float64)
X_test_res = torch.from_numpy(X_test_res_np).unsqueeze(1).to(device, dtype=torch.float64)
y_train_res = torch.from_numpy(y_train_res_np).to(device, dtype=torch.float64)
y_test_res = torch.from_numpy(y_test_res_np).to(device, dtype=torch.float64)

running on GPU
TRAIN deterministic algorithms


In [20]:
def validation(model, validation_subjects, mem_cap_data, X_train, y_train, X_test, y_test):
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
    
    val_mae_loss = np.zeros(args.num_timepoints - 1)
    val_tp_loss = np.zeros(args.num_timepoints - 1)
    mem_cap = np.zeros(args.num_timepoints - 1)
    predicted = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    actual = np.zeros((validation_subjects.shape[0], args.num_timepoints - 1, 35, 35))
    
    model.eval()

    with torch.no_grad():
        for n_subject, data in enumerate(validation_subjects):
            input = data[0]
            for t in range(args.num_timepoints - 1):
                pred = model(input)
                val_mae_loss[t] += mael(pred, data[t + 1])
                val_tp_loss[t] += tp(pred.sum(dim=-1), data[t + 1].sum(dim=-1))
                input = pred
                
                pred_mem_cap = get_mem_cap_from_model(model, pred, 
                                                      X_train, y_train, X_test, y_test)
                actual_mem_cap = torch.tensor(mem_cap_data[n_subject, t + 1]).to(device)
                mem_cap[t] += torch.abs(pred_mem_cap - actual_mem_cap)

                predicted[n_subject, t] = pred.cpu().detach().numpy()
                actual[n_subject, t] = data[t + 1].cpu().detach().numpy()
                

    avg_val_mae_loss = val_mae_loss/len(validation_subjects)
    avg_val_tp_loss = val_tp_loss/len(validation_subjects)
    avg_val_mae_mem_cap = mem_cap/len(validation_subjects)

    return avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, predicted, actual

In [26]:
torch.manual_seed(1)
input_weights = (torch.rand((35, 1), dtype=torch.float64) * 2.0 - 1.0).to(device)

indexes = range(args.num_folds)
kfold = KFold(n_splits=args.num_folds, shuffle=True, random_state=manual_seed)
dataset = dataset.to(device)
actual_mem_caps = np.load('datasets/mem_caps_2.npy')
f = 0

for train, test in kfold.split(range(dataset.shape[0])):
    print(
            f'------------------------------------Fold [{f + 1}/{args.num_folds}]-----------------------------------------')
    
    train_data = dataset[train]
    test_data = dataset[test]
    train_mem_cap = actual_mem_caps[train]
    test_mem_cap = actual_mem_caps[test]
    
    validation_split = int(0.8 * len(train_data))
    train_subjects = train_data[:validation_split]
    train_mem_cap_subjects = train_mem_cap[:validation_split]
    validation_subjects = train_data[validation_split:]
    validation_mem_cap_subjects = train_mem_cap[:validation_split]

    model = GNN_1(device=device, input_weights=input_weights).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=args.lr)
    mael = torch.nn.L1Loss().to(device)
    tp = torch.nn.MSELoss().to(device)
  
    # Start measuring the epochs time
    epochs_start = time.time()
    for epoch in range(args.num_epochs):
        
        print(f'Epoch [{epoch + 1}/{args.num_epochs}]')
        # Set the model in training mode
        model.train()
        
        # this is our loss for all the data
        mae_loss_overall = []
        tp_loss_overall = []
        mae_mem_cap_overall = []
        
        # loop through the data batches
        for data_id, data in enumerate(train_subjects):

            # zero the gradients
            optimizer.zero_grad()
            mae_loss = 0
            tp_loss = 0
            mem_cap_loss = 0
         
            # loop through the time dependent adj matrices in the batches
            for t in range(args.num_timepoints - 1):
                
                pred, output_sig = model(data[t], X_train_res, y_train_res, X_test_res)
                    
                real = data[t + 1]
    
                mae_loss += mael(pred, real)

                # Topological Loss
                tp_loss += tp(pred.sum(dim=-1), real.sum(dim=-1))
                
                # MAE between predicted graph's mem cap and actual graph's mem cap
                predicted_mem_cap = compute_memory_capacity_vectorized(output_sig, y_test_res)
                actual_mem_cap = torch.tensor(train_mem_cap_subjects[data_id, t + 1], requires_grad=True).to(device)
                mem_cap_loss += mael(predicted_mem_cap, actual_mem_cap)

    
            # Calculate the total MAE Loss for the current batch
            mae_loss = mae_loss / (args.num_timepoints - 1)

            # Calculate the total TP Loss for the current batch
            tp_loss = tp_loss / (args.num_timepoints - 1)

            # Calculate the total MAE between Mem Cap Loss for the current batch
            mem_cap_loss = mem_cap_loss / (args.num_timepoints - 1)
            
            # Append to the total MAE Loss
            mae_loss_overall.append(mae_loss.item())
            tp_loss_overall.append(tp_loss.item())
            mae_mem_cap_overall.append(mem_cap_loss.item())
            
            total_loss = mae_loss + args.memcap_coef * mem_cap_loss
            # Update the weights of the neural network
            total_loss.backward()
            optimizer.step()

        mae_loss_overall = np.mean(np.array(mae_loss_overall))
        tp_loss_overall = np.mean(np.array(tp_loss_overall))
        mae_mem_cap_overall = np.mean(np.array(mae_mem_cap_overall))
        print(f"[Train] MAE Loss: {mae_loss_overall}, TP Loss: {tp_loss_overall}, MAE of Mem Caps Loss: {mae_mem_cap_overall}")
    
        avg_val_mae_loss, avg_val_tp_loss, avg_val_mae_mem_cap, _, _ = validation(model, validation_subjects, validation_mem_cap_subjects,
                                                                                  X_train_res, y_train_res, X_test_res, y_test_res)
        print(f"[Validate] MAE Loss Across Timepoints: {avg_val_mae_loss}")
        print(f"[Validate] TP Loss Across Timepoints: {avg_val_tp_loss}")
        print(f"[Validate] MAE of Mem Caps Across Timepoints: {avg_val_mae_mem_cap}")

    
    epochs_end = time.time() - epochs_start
    print()
    print(f'epochs finished with time:{epochs_end}')
    print()
    process = psutil.Process(os.getpid())
    print(f"Current memory usage: {process.memory_info().rss / 1024 ** 2:.2f} MB")
    print()
    
    avg_test_mae_loss, avg_test_tp_loss, avg_test_mem_cap, predicted, original = validation(model, test_data, test_mem_cap,
                                                                                            X_train_res, y_train_res, X_test_res, y_test_res)
    print(f"[Test] MAE Loss Across Timepoints: {avg_test_mae_loss}")
    print(f"[Test] TP Loss Across Timepoints: {avg_test_tp_loss}")
    print(f"[Test] MAE of Mem Caps Across Timepoints: {avg_test_mem_cap}")
    np.save(args.save_path+f"test_mae_losses/mae_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_tp_losses/tp_test_loss_fold_{f}", avg_test_mae_loss)
    np.save(args.save_path+f"test_memcap_losses/memcap_test_loss_fold_{f}", avg_test_mem_cap)
    np.save(args.save_path+f"test_predicted/predicted_fold_{f}", predicted)
    np.save(args.save_path+f"test_original/original_fold_{f}", original)

    torch.save(model.state_dict(),
               args.save_path +f'trained_models/model_fold_{f}')
    f += 1
    

------------------------------------Fold [1/5]-----------------------------------------
Epoch [1/20]
[Train] MAE Loss: 0.03707144242071081, TP Loss: 3.508642232744023, MAE of Mem Caps Loss: 0.2915014259763035
[Validate] MAE Loss Across Timepoints: [0.04091657 0.04338468]
[Validate] TP Loss Across Timepoints: [1.13029826 1.57897723]
[Validate] MAE of Mem Caps Across Timepoints: [0.27634506 0.62171657]
Epoch [2/20]
[Train] MAE Loss: 0.026529635913902894, TP Loss: 0.718762926524505, MAE of Mem Caps Loss: 0.21003925708391546
[Validate] MAE Loss Across Timepoints: [0.03837866 0.04055791]
[Validate] TP Loss Across Timepoints: [1.01133311 1.33522296]
[Validate] MAE of Mem Caps Across Timepoints: [0.27531138 0.64003869]
Epoch [3/20]
[Train] MAE Loss: 0.025520707815303467, TP Loss: 0.6762160551734269, MAE of Mem Caps Loss: 0.17943341783704286
[Validate] MAE Loss Across Timepoints: [0.03778297 0.04009748]
[Validate] TP Loss Across Timepoints: [0.9805401  1.29271722]
[Validate] MAE of Mem Caps Ac

[Validate] MAE Loss Across Timepoints: [0.03781157 0.04060908]
[Validate] TP Loss Across Timepoints: [0.9626565  1.26418066]
[Validate] MAE of Mem Caps Across Timepoints: [0.24267667 0.49905352]
Epoch [6/20]
[Train] MAE Loss: 0.02565018278255593, TP Loss: 0.7012145915068686, MAE of Mem Caps Loss: 0.14935659977326493
[Validate] MAE Loss Across Timepoints: [0.03756969 0.04025608]
[Validate] TP Loss Across Timepoints: [0.95496166 1.21965861]
[Validate] MAE of Mem Caps Across Timepoints: [0.25260803 0.50691601]
Epoch [7/20]
[Train] MAE Loss: 0.025506245263386518, TP Loss: 0.6995925593655556, MAE of Mem Caps Loss: 0.1515105808080516
[Validate] MAE Loss Across Timepoints: [0.03740263 0.04012098]
[Validate] TP Loss Across Timepoints: [0.95885867 1.22674716]
[Validate] MAE of Mem Caps Across Timepoints: [0.24986475 0.5017409 ]
Epoch [8/20]
[Train] MAE Loss: 0.025312214449513704, TP Loss: 0.6942429153714329, MAE of Mem Caps Loss: 0.15779332393261986
[Validate] MAE Loss Across Timepoints: [0.036

[Train] MAE Loss: 0.02394853564328514, TP Loss: 0.6182166746584699, MAE of Mem Caps Loss: 0.17315191028651114
[Validate] MAE Loss Across Timepoints: [0.03636267 0.03720339]
[Validate] TP Loss Across Timepoints: [0.96403456 1.16950428]
[Validate] MAE of Mem Caps Across Timepoints: [0.28325746 0.50497992]
Epoch [11/20]
[Train] MAE Loss: 0.023872714300523512, TP Loss: 0.6180718176765367, MAE of Mem Caps Loss: 0.1710770323860284
[Validate] MAE Loss Across Timepoints: [0.03630058 0.03716778]
[Validate] TP Loss Across Timepoints: [0.96507257 1.17058563]
[Validate] MAE of Mem Caps Across Timepoints: [0.27837828 0.50346921]
Epoch [12/20]
[Train] MAE Loss: 0.02380338220973499, TP Loss: 0.6172150245402008, MAE of Mem Caps Loss: 0.16727103041217095
[Validate] MAE Loss Across Timepoints: [0.03622915 0.03701572]
[Validate] TP Loss Across Timepoints: [0.96241122 1.15480375]
[Validate] MAE of Mem Caps Across Timepoints: [0.26910844 0.50648161]
Epoch [13/20]
[Train] MAE Loss: 0.023747213432216085, TP 

[Train] MAE Loss: 0.023932820724439807, TP Loss: 0.6357814292423427, MAE of Mem Caps Loss: 0.15613954874577424
[Validate] MAE Loss Across Timepoints: [0.03551    0.03754488]
[Validate] TP Loss Across Timepoints: [0.87702101 1.17298222]
[Validate] MAE of Mem Caps Across Timepoints: [0.26593406 0.63441426]
Epoch [16/20]
[Train] MAE Loss: 0.02388371191045735, TP Loss: 0.6359179597347975, MAE of Mem Caps Loss: 0.15677859148125256
[Validate] MAE Loss Across Timepoints: [0.0354106  0.03735739]
[Validate] TP Loss Across Timepoints: [0.87694407 1.15786397]
[Validate] MAE of Mem Caps Across Timepoints: [0.26290471 0.64523321]
Epoch [17/20]
[Train] MAE Loss: 0.023828690958907828, TP Loss: 0.6338146571069956, MAE of Mem Caps Loss: 0.14996698628957014
[Validate] MAE Loss Across Timepoints: [0.03540295 0.03738119]
[Validate] TP Loss Across Timepoints: [0.87624639 1.16096866]
[Validate] MAE of Mem Caps Across Timepoints: [0.23488938 0.60337412]
Epoch [18/20]
[Train] MAE Loss: 0.023828729186789133, T

[Train] MAE Loss: 0.023372006413410418, TP Loss: 0.6090615567518398, MAE of Mem Caps Loss: 0.14187208894661382
[Validate] MAE Loss Across Timepoints: [0.03426684 0.03655614]
[Validate] TP Loss Across Timepoints: [0.85983974 1.13774872]
[Validate] MAE of Mem Caps Across Timepoints: [0.24451119 0.52904664]

epochs finished with time:1819.3014245033264

Current memory usage: 2621.08 MB

[Test] MAE Loss Across Timepoints: [0.03688241 0.0375134 ]
[Test] TP Loss Across Timepoints: [0.95847626 1.18116455]
[Test] MAE of Mem Caps Across Timepoints: [0.26133065 0.52405778]
